# Backtesting with NautilusTrader

---

In this notebook, I'll experiment with a simple backtesting strategy using the **`NautilusTrader`** library. The strategy will be tested on **Bitcoin/USD 1-hour interval data**. This workflow will demonstrate how to set up, implement, and evaluate a trading strategy using historical data.

## Low-level API

The low-level API centers around a `BacktestEngine`, where inputs are initialized and added manually via a Python script. An instantiated `BacktestEngine` can accept the following:

* Lists of `Data` objects, which are automatically sorted into monotonic order based on `ts_init`.
* Multiple venues, manually initialized.
* Multiple actors, manually initialized and added.
* Multiple execution algorithms, manually initialized and added.

This approach offers detailed control over the backtesting process, allowing you to manually configure each component.

In [1]:
import os
import sys
# Add src folder to Python path
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

## Load data

In [2]:
import polars as pl
from src.utils.utils import polars_to_pandas
# from src.config import config

In [3]:
df = pl.read_parquet("../data/BTCUSDT_1h_tmp_365days.parquet")
df.head()

open_time,open,high,low,close,volume,close_time,quote_asset_volume,trades,taker_base_vol,taker_quote_vol,ignore
datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,str
2024-07-16 11:00:00,63443.55,63872.2,63146.45,63746.11,1428.01869,2024-07-16 11:59:59.999,9.0796e7,81332,802.68075,5.1057e7,"""0"""
2024-07-16 12:00:00,63746.11,64265.33,63601.02,63743.31,1785.10099,2024-07-16 12:59:59.999,1.1409e8,101202,928.07253,5.9323e7,"""0"""
2024-07-16 13:00:00,63743.3,64100.0,63400.01,63419.99,1760.41084,2024-07-16 13:59:59.999,1.1229e8,111397,817.08425,5.2132e7,"""0"""
2024-07-16 14:00:00,63419.99,63898.71,63063.83,63836.09,1780.20422,2024-07-16 14:59:59.999,1.1300e8,119653,879.26966,5.5836e7,"""0"""
2024-07-16 15:00:00,63836.09,64960.86,63799.0,64878.24,3605.09775,2024-07-16 15:59:59.999,2.3269e8,195247,2100.49827,1.3557e8,"""0"""


### POC design — Data ingest

In [4]:
from __future__ import annotations
import polars as pl
from pathlib import Path
from nautilus_trader.model import BarType
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from decimal import Decimal

In [5]:
CATALOG_PATH = Path.cwd() / "catalog"
CATALOG_PATH.mkdir(parents=True, exist_ok=True)

# Create a catalog instance
catalog = ParquetDataCatalog(CATALOG_PATH)

Create instrument

In [6]:
BTCUSD = TestInstrumentProvider.btcusdt_binance()
print(BTCUSD)

CurrencyPair(id=BTCUSDT.BINANCE, raw_symbol=BTCUSDT, asset_class=CRYPTOCURRENCY, instrument_class=SPOT, quote_currency=USDT, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=6, size_increment=0.000001, multiplier=1, lot_size=None, margin_init=0, margin_maint=0, maker_fee=0.001, taker_fee=0.001, info=None)


OBLIGATORIO: Pandas df con timestamp como indice

In [7]:
df = df.with_columns(
        (pl.col("open_time") + pl.duration(hours=1)).alias("timestamp"),
    ).to_pandas()

# Change order of columns
df = df.reindex(columns=["timestamp", "open", "high", "low", "close", "volume"])
df = df.set_index("timestamp")
df.head()

,open,high,low,close,volume
timestamp,,,,,
2024-07-16 12:00:00,63443.55,63872.20,63146.45,63746.11,1428.01869
2024-07-16 13:00:00,63746.11,64265.33,63601.02,63743.31,1785.10099
2024-07-16 14:00:00,63743.30,64100.00,63400.01,63419.99,1760.41084
2024-07-16 15:00:00,63419.99,63898.71,63063.83,63836.09,1780.20422
2024-07-16 16:00:00,63836.09,64960.86,63799.00,64878.24,3605.09775


In [8]:
EURUSD_SPOT_1HOUR_BARTYPE = BarType.from_str(
    f"{BTCUSD.id}-1-HOUR-LAST-EXTERNAL"
)

In [9]:
wrangler = BarDataWrangler(bar_type=EURUSD_SPOT_1HOUR_BARTYPE, instrument=BTCUSD)
BARS = wrangler.process(df)

## Simple bt

In [10]:
from nautilus_trader.model.currencies import BTC
from nautilus_trader.model.currencies import USDT

Configure Engine:

The BacktestDataConfig objects are integrated into the backtesting framework through BacktestRunConfig:
(https://nautilustrader.io/docs/latest/concepts/data/#integration-with-backtestrunconfig)

In [ ]:
from nautilus_trader.adapters.binance import BINANCE_VENUE
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.identifiers import TraderId
from nautilus_trader.model.objects import Money

# Configure backtest engine
engine_config = BacktestEngineConfig(
    trader_id=TraderId("BACKTESTER-ENGINE-001"),
    logging=LoggingConfig(log_level="INFO"),
)

# Build the backtest engine
engine = BacktestEngine(config=engine_config)

# Add Venue
instrument = BTCUSD

engine.add_venue(
    venue=BINANCE_VENUE,
    oms_type=OmsType.NETTING,
    account_type=AccountType.CASH,
    base_currency=None,
    starting_balances=[Money(1_000_000.0, USDT), Money(1.0, BTC)],
)

2025-07-31T15:01:36.230700000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: =================================================================
2025-07-31T15:01:36.230709000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-07-31T15:01:36.230710000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-07-31T15:01:36.230710001Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine:  Copyright (C) 2015-2025. All rights reserved.
2025-07-31T15:01:36.230710002Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: =================================================================
2025-07-31T15:01:36.230710003Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: 
2025-07-31T15:01:36.230710004Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-07-31T15:01:36.230710005Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-07-31T15:01:36.230710006Z [INFO] BACKTESTER-ENGINE-001.B

In [12]:
engine.add_instrument(instrument)

bar_type = EURUSD_SPOT_1HOUR_BARTYPE

engine.add_data(BARS)

2025-07-31T15:01:36.245250000Z [INFO] BACKTESTER-ENGINE-001.SimulatedExchange(BINANCE): Added instrument BTCUSDT.BINANCE and created matching engine
2025-07-31T15:01:36.245254000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: Added BTCUSDT.BINANCE Instrument
2025-07-31T15:01:36.248728000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: Added 8_760 BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL Bar elements


In [13]:
from src.strategies.nautilus import SmaCrossConfig, SmaCrossNT

strategy_config = SmaCrossConfig(
    strategy_id="SMA-CROSS-LONG-001",
    instrument_id=BTCUSD.id,
    bar_type=EURUSD_SPOT_1HOUR_BARTYPE,
    trade_size=Decimal("1"),
    fast_sma_period = 30,
    slow_sma_period = 100
    )

# Instantiate and add your strategy
strategy = SmaCrossNT(strategy_config)
engine.add_strategy(strategy=strategy)

2025-07-31T15:01:36.267894000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: READY
2025-07-31T15:01:36.267965000Z [INFO] BACKTESTER-ENGINE-001.ExecEngine: Registered OMS.UNSPECIFIED for Strategy SMA-000
2025-07-31T15:01:36.267971000Z [INFO] BACKTESTER-ENGINE-001.BACKTESTER-ENGINE-001: Registered Strategy SMA-000


In [ ]:
# Run the engine (from start to end of data)
engine.run()

2025-07-31T15:01:36.273375000Z [INFO] BACKTESTER-ENGINE-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=CASH, base_currency=None, is_reported=True, balances=[AccountBalance(total=1_000_000.00000000 USDT, locked=0.00000000 USDT, free=1_000_000.00000000 USDT), AccountBalance(total=1.00000000 BTC, locked=0.00000000 BTC, free=1.00000000 BTC)], margins=[], event_id=71a2baaf-e9c0-4350-8bd3-0af092905472)
2024-07-16T12:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: STARTING
2024-07-16T12:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.DataClient-BINANCE: Connecting...
2024-07-16T12:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.DataClient-BINANCE: Connected
2024-07-16T12:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.DataClient-BINANCE: RUNNING
2024-07-16T12:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.DataEngine: RUNNING
2024-07-16T12:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.RiskEngine: RUNNING
2024-07-16T12:00:00.000000000Z [INFO] BACKTESTER-ENGINE-0

2024-07-16T12:00:00.000000000Z [ERROR] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Received <Bar[0]> data for unknown bar type


0.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,66755.42,66804.00,66417.66,66787.99,1417.550760,1721415600000000000)
2024-07-19T19:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Waiting for indicators to warm up [80]
2024-07-19T20:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,66788.00,67386.00,66787.99,67157.26,3785.527120,1721419200000000000)
2024-07-19T20:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Waiting for indicators to warm up [81]
2024-07-19T21:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,67157.25,67221.17,66880.39,66931.80,1328.523920,1721422800000000000)
2024-07-19T21:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Waiting for indicators to warm up [82]
2024-07-19T22:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT

188.01,951.331040,1721610000000000000)
2024-07-22T02:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,68188.01,68228.01,67762.00,67850.01,847.396290,1721613600000000000)
2024-07-22T03:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,67850.01,68199.31,67753.72,68070.62,577.493380,1721617200000000000)
2024-07-22T04:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,68070.63,68102.00,67836.62,67880.01,554.243380,1721620800000000000)
2024-07-22T05:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,67880.01,67996.01,67760.00,67923.98,398.156190,1721624400000000000)
2024-07-22T06:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,67923.99,67940.01,67520.00,67626.01,733.176930,1721628000000000000)
2024-07-22T07:00

In [15]:
account_report = pl.DataFrame(engine.trader.generate_account_report(BINANCE_VENUE))
orders_report = pl.DataFrame(engine.trader.generate_order_fills_report())
positions_report = pl.DataFrame(engine.trader.generate_positions_report())

In [16]:
account_report

total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
str,str,str,str,str,str,str,list[null],bool,struct[0]
"""1000000.00000000""","""0E-8""","""1000000.00000000""","""USDT""","""BINANCE-001""","""CASH""",null,[],true,{}
"""1.00000000""","""0E-8""","""1.00000000""","""BTC""","""BINANCE-001""","""CASH""",null,[],true,{}
"""933343.41000000""","""0E-8""","""933343.41000000""","""USDT""","""BINANCE-001""","""CASH""",null,[],false,{}
"""2.00000000""","""0E-8""","""2.00000000""","""BTC""","""BINANCE-001""","""CASH""",null,[],false,{}
"""999137.55000000""","""0E-8""","""999137.55000000""","""USDT""","""BINANCE-001""","""CASH""",null,[],false,{}
…,…,…,…,…,…,…,…,…,…
"""1.00000000""","""0E-8""","""1.00000000""","""BTC""","""BINANCE-001""","""CASH""",null,[],false,{}
"""908699.64762000""","""0E-8""","""908699.64762000""","""USDT""","""BINANCE-001""","""CASH""",null,[],false,{}
"""2.00000000""","""0E-8""","""2.00000000""","""BTC""","""BINANCE-001""","""CASH""",null,[],false,{}


In [17]:
orders_report

trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,time_in_force,is_reduce_only,is_quote_quantity,filled_qty,liquidity_side,avg_px,slippage,commissions,emulation_trigger,status,contingency_type,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
str,str,str,str,str,str,str,str,str,str,str,bool,bool,str,str,f64,f64,list[str],str,str,str,str,str,str,str,str,str,str,str,"datetime[ns, UTC]","datetime[ns, UTC]"
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-001""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-101""","""MARKET""","""BUY""","""1.000000""","""IOC""",false,false,"""1.000000""","""TAKER""",66590.0,0.0,"[""66.59000000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""f66e0636-7ad4-483a-9868-6c66e2…",2024-07-20 15:00:00 UTC,2024-07-20 15:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-002""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-178""","""MARKET""","""SELL""","""1.000000""","""GTC""",true,false,"""1.000000""","""TAKER""",65860.0,0.0,"[""65.86000000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""56a3467c-6199-4001-8836-b400f3…",2024-07-23 19:00:00 UTC,2024-07-23 19:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-003""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-250""","""MARKET""","""BUY""","""1.000000""","""IOC""",false,false,"""1.000000""","""TAKER""",67372.0,0.0,"[""67.37200000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""9eff1c2b-e025-42d8-95a0-063f73…",2024-07-26 18:00:00 UTC,2024-07-26 18:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-004""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-339""","""MARKET""","""SELL""","""1.000000""","""GTC""",true,false,"""1.000000""","""TAKER""",66608.01,0.0,"[""66.60801000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""37055cd5-c3b0-4188-9f7a-591fd2…",2024-07-30 10:00:00 UTC,2024-07-30 10:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-005""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-551""","""MARKET""","""BUY""","""1.000000""","""IOC""",false,false,"""1.000000""","""TAKER""",56765.34,0.0,"[""56.76534000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""131de71a-806e-49f0-9dfa-c45d27…",2024-08-08 05:00:00 UTC,2024-08-08 05:00:00 UTC
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-100""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-8598""","""MARKET""","""SELL""","""1.000000""","""GTC""",true,false,"""1.000000""","""TAKER""",108148.57,0.0,"[""108.14857000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""ff84d843-d078-471a-ad67-befb6d…",2025-07-05 13:00:00 UTC,2025-07-05 13:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-101""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-8643""","""MARKET""","""BUY""","""1.000000""","""IOC""",false,false,"""1.000000""","""TAKER""",109011.7,0.0,"[""109.01170000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""c0b2e149-b680-4d8f-90ee-220e66…",2025-07-07 09:00:00 UTC,2025-07-07 09:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-102""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-8674""","""MARKET""","""SELL""","""1.000000""","""GTC""",true,false,"""1.000000""","""TAKER""",108376.0,0.0,"[""108.37

In [18]:
positions_report

trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
str,str,str,str,str,str,str,str,str,str,i64,"datetime[ns, UTC]",i64,"datetime[ns, UTC]",i64,f64,f64,list[str],f64,str
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20240720-150000-001-000-1""","""O-20240723-190000-001-000-2""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1721487600000000000,2024-07-20 15:00:00 UTC,1721761200000000000,2024-07-23 19:00:00 UTC,273600000000000,66590.0,65860.0,"[""132.45000000 USDT""]",-0.01096,"""-862.45000000 USDT"""
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20240726-180000-001-000-3""","""O-20240730-100000-001-000-4""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1722016800000000000,2024-07-26 18:00:00 UTC,1722333600000000000,2024-07-30 10:00:00 UTC,316800000000000,67372.0,66608.01,"[""133.98001000 USDT""]",-0.01134,"""-897.97001000 USDT"""
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20240808-050000-001-000-5""","""O-20240812-070000-001-000-6""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1723093200000000000,2024-08-08 05:00:00 UTC,1723446000000000000,2024-08-12 07:00:00 UTC,352800000000000,56765.34,58403.76,"[""115.16910000 USDT""]",0.02886,"""1523.25090000 USDT"""
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20240814-070000-001-000-7""","""O-20240815-080000-001-000-8""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1723618800000000000,2024-08-14 07:00:00 UTC,1723708800000000000,2024-08-15 08:00:00 UTC,90000000000000,60861.5,58060.37,"[""118.92187000 USDT""]",-0.04602,"""-2920.05187000 USDT"""
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20240817-210000-001-000-9""","""O-20240819-190000-001-000-10""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1723928400000000000,2024-08-17 21:00:00 UTC,1724094000000000000,2024-08-19 19:00:00 UTC,165600000000000,59397.12,59011.99,"[""118.40911000 USDT""]",-0.00648,"""-503.53911000 USDT"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20250616-090000-001-000-95""","""O-20250618-030000-001-000-96""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1750064400000000000,2025-06-16 09:00:00 UTC,1750215600000000000,2025-06-18 03:00:00 UTC,151200000000000,107180.0,104856.06,"[""212.03606000 USDT""]",-0.02168,"""-2535.97606000 USDT"""
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20250624-070000-001-000-97""","""O-20250701-100000-001-000-98""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1750748400000000000,2025-06-24 07:00:00 UTC,1751364000000000000,2025-07-01 10:00:00 UTC,615600000000000,105655.23,106556.39,"[""212.21162000 USDT""]",0.00853,"""688.94838000 USDT"""
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20250703-010000-001-000-99""","""O-20250705-130000-001-000-100""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1751504400000000000,2025-07-03 01:00:00 UTC,1751720400000000000,2025-07-05 13:00:00 UTC,216000000000000,108915.0,108148.57,"[""217.06357000 USDT""]",-0.00704,"""-983.49357000 USDT"""


In [19]:
# For repeated backtest runs make sure to reset the engine
engine.reset()

# Good practice to dispose of the object
engine.dispose()

2025-07-31T15:01:45.865667000Z [INFO] BACKTESTER-ENGINE-001.DataClient-BINANCE: READY
2025-07-31T15:01:45.865691000Z [INFO] BACKTESTER-ENGINE-001.DataEngine: READY
2025-07-31T15:01:45.865711000Z [INFO] BACKTESTER-ENGINE-001.ExecClient-BINANCE: READY
2025-07-31T15:01:45.865949000Z [INFO] BACKTESTER-ENGINE-001.Cache: Reset
2025-07-31T15:01:45.865958000Z [INFO] BACKTESTER-ENGINE-001.ExecEngine: READY
2025-07-31T15:01:45.865970000Z [INFO] BACKTESTER-ENGINE-001.RiskEngine: READY
2025-07-31T15:01:45.865984000Z [INFO] BACKTESTER-ENGINE-001.OrderEmulator: READY
2025-07-31T15:01:45.866012000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: READY
2025-07-31T15:01:45.866172000Z [INFO] BACKTESTER-ENGINE-001.Portfolio: READY
2025-07-31T15:01:45.866184000Z [INFO] BACKTESTER-ENGINE-001.BACKTESTER-ENGINE-001: READY
2025-07-31T15:01:45.866262000Z [INFO] BACKTESTER-ENGINE-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=CASH, base_currency=None, is_reported=True, balances=[Accou